In [1]:
import pandas as pd
import glob

In [6]:
glob.glob('./sample_preprocessing_output/*')

['./sample_preprocessing_output\\wsj_0001.tsv',
 './sample_preprocessing_output\\wsj_0002.tsv',
 './sample_preprocessing_output\\wsj_0003.tsv',
 './sample_preprocessing_output\\wsj_0004.tsv',
 './sample_preprocessing_output\\wsj_0005.tsv',
 './sample_preprocessing_output\\wsj_0006.tsv',
 './sample_preprocessing_output\\wsj_0007.tsv',
 './sample_preprocessing_output\\wsj_0008.tsv',
 './sample_preprocessing_output\\wsj_0009.tsv',
 './sample_preprocessing_output\\wsj_0010.tsv']

In [59]:
filepath = './sample_preprocessing_output/wsj_0003.tsv'

In [60]:
df = pd.read_csv(filepath, delimiter='\t', index_col=0)
df = df[['sentence_number', 'doc_token_number', 'sentence_token_number', 'token', 'lemma', 'POS', 
    'dependency_head', 'dependency_label', 'ne_info', 'cue_label']]
df

,sentence_number,doc_token_number,sentence_token_number,token,lemma,POS,dependency_head,dependency_label,ne_info,cue_label
0,1,1,1,A,a,DT,2,det,O,0
1,1,2,2,form,form,NN,13,nsubj,O,0
2,1,3,3,of,of,IN,4,case,O,0
3,1,4,4,asbestos,asbesto,NN,2,nmod,O,0
4,1,5,5,once,once,RB,6,advmod,O,0
5,1,6,6,used,use,VBD,35,ccomp,O,0
6,1,7,7,to,to,TO,8,mark,O,0
7,1,8,8,make,make,VB,6,xcomp,O,0
8,1,9,9,Kent,Kent,NNP,11,compound,S-GPE,0
9,1,10,10,cigarette,cigarette,NN,11,compound,O,0


In [61]:
df['near_sentence_boundary'] = 0

In [62]:
def get_boundary_indices(df, boundary_type='sent'):
    indices = []
    if boundary_type = 'sent':
        for index in df[df['sentence_token_number'] == 1].index:
            indices.append(index)
            if index > 0:   # Only if index is not 0 then add previous index to list
                indices.append(index-1)
    # Boundary at start of doc for sent and doc type
    indices.append(0)
    # Also sentence boundary at end of doc
    indices.append(df.shape[0]-1)
    indices.append(df.shape[0]-2)
    return indices

In [63]:
indices = get_sentence_boundary_indices(df)

In [64]:
df.loc[indices, 'near_sentence_boundary'] = 1

In [65]:
df

,sentence_number,doc_token_number,sentence_token_number,token,lemma,POS,dependency_head,dependency_label,ne_info,cue_label,near_sentence_boundary
0,1,1,1,A,a,DT,2,det,O,0,1
1,1,2,2,form,form,NN,13,nsubj,O,0,1
2,1,3,3,of,of,IN,4,case,O,0,0
3,1,4,4,asbestos,asbesto,NN,2,nmod,O,0,0
4,1,5,5,once,once,RB,6,advmod,O,0,0
5,1,6,6,used,use,VBD,35,ccomp,O,0,0
6,1,7,7,to,to,TO,8,mark,O,0,0
7,1,8,8,make,make,VB,6,xcomp,O,0,0
8,1,9,9,Kent,Kent,NNP,11,compound,S-GPE,0,0
9,1,10,10,cigarette,cigarette,NN,11,compound,O,0,0
